In [1]:
# Goal 2 - Predicting Attention (Deep Learning)

In [1]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.2.0


In [3]:
from sklearn.pipeline import Pipeline
from sklearn import linear_model, datasets
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    OneHotEncoder,
    LabelEncoder,
)
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.metrics import accuracy_score


from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, GroupShuffleSplit
from sklearn.metrics import (
    f1_score,
    mean_squared_error,
    r2_score,
    classification_report,
    confusion_matrix,
)

from sklearn import metrics
from numpy import linalg as LA

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
project_dir = "/gdrive/My Drive/Industry Project"
%cd {project_dir}

/gdrive/My Drive/Propulsion/Capstone Project/Group Folder


In [6]:
dataset = pd.read_csv("project_df_final.csv")

dataset.head()

,local_timestamp,headpose_pitch,headpose_roll,headpose_yaw,looking_at_screen,person_id,poi_id,x_world,y_world,z_world,present_in_1,present_in_2,present_in_5,gender,age,total_time_spent_seconds,cumulative_time,dx_world,dy_world,d_time,x_velocity,y_velocity,velocity,distance_to_camera,ema_5_x,ema_5_y,ema_5_velocity,ema_10_x,ema_10_y,ema_10_velocity
0,2020-07-13 09:32:45,344.0,356.0,54.0,1,000adf04-7a12-4af4-9c4d-7117495e9000,444,0.227504,1.251269,0.940917,1,1,1,male,32.0,20,0,0.677319,-0.134942,1.0,0.677319,-0.134942,0.690630,1.271783,0.677319,-0.134942,0.690630,0.677319,-0.134942,0.690630
1,2020-07-13 09:32:46,344.0,356.0,54.0,1,000adf04-7a12-4af4-9c4d-7117495e9000,444,-0.449814,1.386211,0.979368,1,1,1,male,32.0,20,1,0.306883,-0.057870,1.0,0.306883,-0.057870,0.312292,1.457365,0.455057,-0.088699,0.463627,0.473579,-0.092552,0.482544
2,2020-07-13 09:32:47,344.0,356.0,54.0,1,000adf04-7a12-4af4-9c4d-7117495e9000,444,-0.756697,1.444081,1.006110,1,1,1,male,32.0,20,2,0.074132,0.090651,1.0,0.074132,0.090651,0.117103,1.630325,0.274619,-0.003744,0.299484,0.313004,-0.018906,0.335639
3,2020-07-13 09:32:48,344.0,356.0,54.0,1,000adf04-7a12-4af4-9c4d-7117495e9000,444,-0.830829,1.353430,0.976361,1,1,1,male,32.0,20,3,0.182801,-0.304498,1.0,0.182801,-0.304498,0.355155,1.588096,0.236479,-0.128672,0.322609,0.270108,-0.112996,0.342069
4,2020-07-13 09:32:49,344.0,356.0,54.0,1,000adf04-7a12-4af4-9c4d-7117495e9000,444,-1.013630,1.657928,0.961924,1,1,1,male,32.0,20,4,0.279102,-0.393887,1.0,0.279102,-0.393887,0.482747,1.943237,0.252841,-0.230485,0.384084,0.272690,-0.193632,0.382454


In [7]:
#dataset.info()

Add labels to dataframe

In [8]:
def add_labels_in_n(d, n):
    x_col = "x_in_" + str(n)
    y_col = "y_in_" + str(n)
    person_col = "person_in_" + str(n)

    d[x_col] = d["x_world"].shift(-n)
    d[y_col] = d["y_world"].shift(-n)
    d[person_col] = d["person_id"].shift(-n)

    return d[d["person_id"] == d[person_col]]

Create 3 sub dataframes for each prediction time window (1,2,5 seconds)

In [9]:
df_model_5 = add_labels_in_n(dataset, 5)
df_model_2 = add_labels_in_n(dataset, 2)
df_model_1 = add_labels_in_n(dataset, 1)

Define preprocessing pipeline

In [10]:
# Data Preprocessing

# Define numerical features
numerical_features = [
    "x_world",
    "y_world",
    "age",
    "ema_5_x",
    "ema_5_y",
    "ema_5_velocity",
    "cumulative_time",
    "dx_world",
    "dy_world",
    "d_time",
    "x_velocity",
    "y_velocity",
    "velocity",
    "distance_to_camera",
    "ema_10_x",
    "ema_10_y",
    "ema_10_velocity",
]

# Define categorical features
categorical_features = ["gender", "looking_at_screen"]

# Define transformers
numerical_transformer = Pipeline(
    steps=[("scaler", StandardScaler()),]
)

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

# Combine feature transformers
data_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numerical_transformer, numerical_features),
        ("categorical", categorical_transformer, categorical_features),
    ],
    remainder="passthrough",
)  # features not listed in categorical/numerical lists will be ignored ("passthrough")

# Define preprocessing pipeline (data transformation & dimension reduction, not included for now)
preprocessor = Pipeline(steps=[("data_transformer", data_transformer),])

Define functions to split sub dataframes (1,2,5 seconds) by POI

In [11]:
def define_target_and_df_per_window_5(
    sec=5, df=df_model_5, columns=[],
):
    if columns == []:
        columns = [
            "poi_id",
            "person_id",
            "x_world",
            "y_world",
            "age",
            "gender",
            "looking_at_screen",
            "ema_5_x",
            "ema_5_y",
            "ema_5_velocity",
            "cumulative_time",
            "dx_world",
            "dy_world",
            "d_time",
            "x_velocity",
            "y_velocity",
            "velocity",
            "distance_to_camera",
            "ema_10_x",
            "ema_10_y",
            "ema_10_velocity",
            "x_in_5",
            "y_in_5",
        ]

    df_model = df[columns]
    print("The model target window is " + str(sec) + " seconds. \n")
    print(df_model.info())

    df_inst_1 = df_model.loc[df["poi_id"] == 444]
    df_inst_450 = df_model.loc[df["poi_id"] == 450]
    df_inst_453 = df_model.loc[df["poi_id"] == 453]
    df_inst_456 = df_model.loc[df["poi_id"] == 456]
    df_inst_457 = df_model.loc[df["poi_id"] == 457]
    df_inst_466 = df_model.loc[df["poi_id"] == 466]
    df_inst_529 = df_model.loc[df["poi_id"] == 529]

    return (
        df_inst_1,
        df_inst_450,
        df_inst_453,
        df_inst_456,
        df_inst_457,
        df_inst_466,
        df_inst_529,
    )

In [12]:
def define_target_and_df_per_window_2(
    sec=2, df=df_model_2, columns=[],
):
    if columns == []:
        columns = [
            "poi_id",
            "person_id",
            "x_world",
            "y_world",
            "age",
            "gender",
            "looking_at_screen",
            "ema_5_x",
            "ema_5_y",
            "ema_5_velocity",
            "cumulative_time",
            "dx_world",
            "dy_world",
            "d_time",
            "x_velocity",
            "y_velocity",
            "velocity",
            "distance_to_camera",
            "ema_10_x",
            "ema_10_y",
            "ema_10_velocity",
            "x_in_2",
            "y_in_2",
        ]

    df_model = df[columns]
    print("The model target window is " + str(sec) + " seconds. \n")
    print(df_model.info())

    df_inst_1 = df_model.loc[df["poi_id"] == 444]
    df_inst_450 = df_model.loc[df["poi_id"] == 450]
    df_inst_453 = df_model.loc[df["poi_id"] == 453]
    df_inst_456 = df_model.loc[df["poi_id"] == 456]
    df_inst_457 = df_model.loc[df["poi_id"] == 457]
    df_inst_466 = df_model.loc[df["poi_id"] == 466]
    df_inst_529 = df_model.loc[df["poi_id"] == 529]

    return (
        df_inst_1,
        df_inst_450,
        df_inst_453,
        df_inst_456,
        df_inst_457,
        df_inst_466,
        df_inst_529,
    )

In [13]:
def define_target_and_df_per_window_1(
    sec=1, df=df_model_1, columns=[],
):
    if columns == []:
        columns = [
            "poi_id",
            "person_id",
            "x_world",
            "y_world",
            "age",
            "gender",
            "looking_at_screen",
            "ema_5_x",
            "ema_5_y",
            "ema_5_velocity",
            "cumulative_time",
            "dx_world",
            "dy_world",
            "d_time",
            "x_velocity",
            "y_velocity",
            "velocity",
            "distance_to_camera",
            "ema_10_x",
            "ema_10_y",
            "ema_10_velocity",
            "x_in_1",
            "y_in_1",
        ]

    df_model = df[columns]
    print("The model target window is " + str(sec) + " seconds. \n")
    print(df_model.info())

    df_inst_1 = df_model.loc[df["poi_id"] == 444]
    df_inst_450 = df_model.loc[df["poi_id"] == 450]
    df_inst_453 = df_model.loc[df["poi_id"] == 453]
    df_inst_456 = df_model.loc[df["poi_id"] == 456]
    df_inst_457 = df_model.loc[df["poi_id"] == 457]
    df_inst_466 = df_model.loc[df["poi_id"] == 466]
    df_inst_529 = df_model.loc[df["poi_id"] == 529]

    return (
        df_inst_1,
        df_inst_450,
        df_inst_453,
        df_inst_456,
        df_inst_457,
        df_inst_466,
        df_inst_529,
    )

Define function to train-test split per POI and per prediction window (resulting in 7 x 3 dataframes, and 21 x_test, x_train, y_test, y_train)

In [14]:
def split_per_group(d):

    train_inds, test_inds = next(
        GroupShuffleSplit(test_size=0.2, n_splits=2, random_state=42).split(
            d, groups=d["person_id"]
        )
    )

    train = d.iloc[train_inds]
    test = d.iloc[test_inds]

    y_train = train.iloc[:, -2:]
    y_test = test.iloc[:, -2:]

    # Drop present_in_sec, person_id and POI_id from our POI dataframes (not needed and leading to errors)
    X_train = train[train.columns[2:-2]]
    X_test = test[test.columns[2:-2]]

    return X_train, y_train, X_test, y_test


(
    df_inst_444_1,
    df_inst_450_1,
    df_inst_453_1,
    df_inst_456_1,
    df_inst_457_1,
    df_inst_466_1,
    df_inst_529_1,
) = define_target_and_df_per_window_1(1,)

X_train_444_1, y_train_444_1, X_test_444_1, y_test_444_1 = split_per_group(
    df_inst_444_1
)
X_train_450_1, y_train_450_1, X_test_450_1, y_test_450_1 = split_per_group(
    df_inst_450_1
)
X_train_453_1, y_train_453_1, X_test_453_1, y_test_453_1 = split_per_group(
    df_inst_453_1
)
X_train_456_1, y_train_456_1, X_test_456_1, y_test_456_1 = split_per_group(
    df_inst_456_1
)
X_train_457_1, y_train_457_1, X_test_457_1, y_test_457_1 = split_per_group(
    df_inst_457_1
)
X_train_466_1, y_train_466_1, X_test_466_1, y_test_466_1 = split_per_group(
    df_inst_466_1
)
X_train_529_1, y_train_529_1, X_test_529_1, y_test_529_1 = split_per_group(
    df_inst_529_1
)

(
    df_inst_444_2,
    df_inst_450_2,
    df_inst_453_2,
    df_inst_456_2,
    df_inst_457_2,
    df_inst_466_2,
    df_inst_529_2,
) = define_target_and_df_per_window_2(2,)

X_train_444_2, y_train_444_2, X_test_444_2, y_test_444_2 = split_per_group(
    df_inst_444_2
)
X_train_450_2, y_train_450_2, X_test_450_2, y_test_450_2 = split_per_group(
    df_inst_450_2
)
X_train_453_2, y_train_453_2, X_test_453_2, y_test_453_2 = split_per_group(
    df_inst_453_2
)
X_train_456_2, y_train_456_2, X_test_456_2, y_test_456_2 = split_per_group(
    df_inst_456_2
)
X_train_457_2, y_train_457_2, X_test_457_2, y_test_457_2 = split_per_group(
    df_inst_457_2
)
X_train_466_2, y_train_466_2, X_test_466_2, y_test_466_2 = split_per_group(
    df_inst_466_2
)
X_train_529_2, y_train_529_2, X_test_529_2, y_test_529_2 = split_per_group(
    df_inst_529_2
)


(
    df_inst_444_5,
    df_inst_450_5,
    df_inst_453_5,
    df_inst_456_5,
    df_inst_457_5,
    df_inst_466_5,
    df_inst_529_5,
) = define_target_and_df_per_window_5(5,)

X_train_444_5, y_train_444_5, X_test_444_5, y_test_444_5 = split_per_group(
    df_inst_444_5
)
X_train_450_5, y_train_450_5, X_test_450_5, y_test_450_5 = split_per_group(
    df_inst_450_5
)
X_train_453_5, y_train_453_5, X_test_453_5, y_test_453_5 = split_per_group(
    df_inst_453_5
)
X_train_456_5, y_train_456_5, X_test_456_5, y_test_456_5 = split_per_group(
    df_inst_456_5
)
X_train_457_5, y_train_457_5, X_test_457_5, y_test_457_5 = split_per_group(
    df_inst_457_5
)
X_train_466_5, y_train_466_5, X_test_466_5, y_test_466_5 = split_per_group(
    df_inst_466_5
)
X_train_529_5, y_train_529_5, X_test_529_5, y_test_529_5 = split_per_group(
    df_inst_529_5
)

train_test_per_camera_dict = {
    "cam444_1": [X_train_444_1, y_train_444_1, X_test_444_1, y_test_444_1],
    "cam450_1": [X_train_450_1, y_train_450_1, X_test_450_1, y_test_450_1],
    "cam453_1": [X_train_453_1, y_train_453_1, X_test_453_1, y_test_453_1],
    "cam456_1": [X_train_456_1, y_train_456_1, X_test_456_1, y_test_456_1],
    "cam457_1": [X_train_457_1, y_train_457_1, X_test_457_1, y_test_457_1],
    "cam466_1": [X_train_466_1, y_train_466_1, X_test_466_1, y_test_466_1],
    "cam529_1": [X_train_529_1, y_train_529_1, X_test_529_1, y_test_529_1],
    "cam444_2": [X_train_444_2, y_train_444_2, X_test_444_2, y_test_444_2],
    "cam450_2": [X_train_450_2, y_train_450_2, X_test_450_2, y_test_450_2],
    "cam453_2": [X_train_453_2, y_train_453_2, X_test_453_2, y_test_453_2],
    "cam456_2": [X_train_456_2, y_train_456_2, X_test_456_2, y_test_456_2],
    "cam457_2": [X_train_457_2, y_train_457_2, X_test_457_2, y_test_457_2],
    "cam466_2": [X_train_466_2, y_train_466_2, X_test_466_2, y_test_466_2],
    "cam529_2": [X_train_529_2, y_train_529_2, X_test_529_2, y_test_529_2],
    "cam444_5": [X_train_444_5, y_train_444_5, X_test_444_5, y_test_444_5],
    "cam450_5": [X_train_450_5, y_train_450_5, X_test_450_5, y_test_450_5],
    "cam453_5": [X_train_453_5, y_train_453_5, X_test_453_5, y_test_453_5],
    "cam456_5": [X_train_456_5, y_train_456_5, X_test_456_5, y_test_456_5],
    "cam457_5": [X_train_457_5, y_train_457_5, X_test_457_5, y_test_457_5],
    "cam466_5": [X_train_466_5, y_train_466_5, X_test_466_5, y_test_466_5],
    "cam529_5": [X_train_529_5, y_train_529_5, X_test_529_5, y_test_529_5],
}

The model target window is 1 seconds. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1986463 entries, 0 to 2078711
Data columns (total 23 columns):
 #   Column              Dtype  
---  ------              -----  
 0   poi_id              int64  
 1   person_id           object 
 2   x_world             float64
 3   y_world             float64
 4   age                 float64
 5   gender              object 
 6   looking_at_screen   int64  
 7   ema_5_x             float64
 8   ema_5_y             float64
 9   ema_5_velocity      float64
 10  cumulative_time     int64  
 11  dx_world            float64
 12  dy_world            float64
 13  d_time              float64
 14  x_velocity          float64
 15  y_velocity          float64
 16  velocity            float64
 17  distance_to_camera  float64
 18  ema_10_x            float64
 19  ema_10_y            float64
 20  ema_10_velocity     float64
 21  x_in_1              float64
 22  y_in_1              float64
dtypes: float64(18), i

Define a function to build a Neural Net model

In [15]:
def build_model(X_train, y_test):
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(X_train[0])]),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_test.shape[1],) #activation='softmax')
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
               metrics=['mae', 'mse', tf.metrics.RootMeanSquaredError()])
  return model

Define function to run the model

In [16]:
def run_models(X_train, y_train, X_test, y_test):

    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    model = build_model(X_train, y_test)

    X_train_tensor = tf.constant(X_train, dtype = tf.float32, shape=X_train.shape)
    X_test_tensor = tf.constant(X_test, dtype = tf.float32, shape=X_test.shape)

#    model = build_model(X_train_tensor, y_test)   

    fit = model.fit(X_train_tensor, y_train)
    predictions = model.predict(X_test_tensor)
    #score = model.score(X_test_tensor, y_test)

    print(y_test)
    print(predictions)

    print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, predictions))
    print("Mean Squared Error:", metrics.mean_squared_error(y_test, predictions))
    print(
        "Root Mean Squared Error:",
        np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    )

    r2 = metrics.r2_score(y_test, predictions, multioutput="variance_weighted")
    n, p = X_test.shape

    print("R Square Score:", r2)

    ad_r_square = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

    print("Adjusted R Square Score:", ad_r_square)

    # calculate euclidian distance between predicted and actual (x,y) points: 
    val = []
    for i in range(len(y_test)):
        val.append(LA.norm(y_test.values[i] - predictions[i]))

    print("euclidian distance between pred and actual: " + str(np.mean(val)))

    return y_test, predictions

Run model

In [17]:
dict1 = {
    "cam444_1": [X_train_444_1, y_train_444_1, X_test_444_1, y_test_444_1],
}

for key, value in train_test_per_camera_dict.items():
#for key, value in dict1.items():
    print("\n-----", key)
    test, preds = run_models(*value)


----- cam444_1
3722/3722 [==============================] - 11s 3ms/step - loss: 0.1169 - mae: 0.1199 - mse: 0.1169 - root_mean_squared_error: 0.3419
          x_in_1     y_in_1
185    -1.332999  10.809081
186    -1.063615  10.019430
187    -1.029271   9.627491
188    -1.140910   9.322717
189    -0.966482   9.534516
...          ...        ...
155070  1.692269   3.544731
155071  1.608889   3.226548
155072  1.559912   2.063911
155073  1.687831   1.054498
155074  1.800506   1.054498

[29763 rows x 2 columns]
[[-1.3361579 10.759259 ]
 [-1.1135302  9.999301 ]
 [-1.0620084  9.585326 ]
 ...
 [ 1.538938   2.1505606]
 [ 1.6886127  0.9769608]
 [ 1.832799   0.9454748]]
Mean Absolute Error: 0.06226000635707583
Mean Squared Error: 0.008394687021472563
Root Mean Squared Error: 0.09162252464035557
R Square Score: 0.9986297731510514
Adjusted R Square Score: 0.9986288056394066
euclidian distance between pred and actual: 0.09962334471627458
